In [1]:
from clamv import TestDataGen, Resizer, FormatConverter, MultiDimensionalModel
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
import numpy as np

# Data

In [2]:
resize = Resizer(50, 60)
rgb2yuv = FormatConverter(source='RGB', dest='YUV', flip_axis=True)

In [3]:
data = [(rgb2yuv(resize(x)), y) for x, y in TestDataGen(path='../recognition-pipeline/data', numeric_y=True)]
X, y = zip(*data)
X = np.array(X).reshape(len(X), 3, -1)
y = np.array(y)

# Test

In [5]:
from sklearn.model_selection import cross_val_score

# EigenFaces

In [6]:
EigenFaces = Pipeline([
    ('eigenfaces', MultiDimensionalModel(PCA(n_components=150), dimensions=3)),
    ('classify', SVC(kernel='linear', class_weight='balanced'))
])

In [7]:
%time cross_val_score(EigenFaces, X, y, cv=3)

Wall time: 31.3 s


array([ 0.76093943,  0.85481078,  0.595692  ])

# FisherFaces

In [10]:
FisherFaces = Pipeline([
    ('fisherfaces', MultiDimensionalModel(LDA(n_components=150), dimensions=3)),
    ('classify', SVC(kernel='linear', class_weight='balanced'))
])

In [11]:
%time cross_val_score(FisherFaces, X, y, cv=3)

Wall time: 2min 28s


array([ 0.62941904,  0.76255256,  0.53107205])